In [1]:
import numpy
import spacy
import gensim
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models import Word2Vec
import os
import io
import unicodedata
import multiprocessing
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
def get_data(dirname):
    """ Collects all files in the given folder, extract each line of every file as an element, concatenate the results  """
    if not os.listdir(dirname):
        print("Files not found-Empty Directory ")
        return
    else:
        files = os.listdir(dirname) # file names i.e : cv file , job offer file
    filenames = [dirname+"/"+files[i] for i in range(len(files))] # put file paths in a list
    #train_data = [io.open(filenames[i], 'r', encoding='latin-1').read() for i in range(len(filenames))]
    train_data=[]
    for file in filenames:
        with open(file,errors='ignore') as f:
            line_list=[]
            for l in f:
                line = l.rstrip('\n')
                line_uni = unicodedata.normalize("NFKD",line)
                line_list.append(line_uni)
        #lines = [line.rstrip('\n') for line in open(file)]
        train_data= train_data + line_list
    return train_data

def _keep_token(t):
    return (t.is_alpha and 
            not (t.is_space or t.is_punct or 
                 t.is_stop or t.like_num))
def _lemmatize_doc(doc):
    return [ t.lemma_ for t in doc if _keep_token(t)]

def spacy_preproc(document):
    """ preprocess the documents using spacy """
    process_result=[]
    for doc in document:
         process_result.append(_lemmatize_doc(doc))
    return process_result

def gensim_preproc(document):
    """ preprocess the documents using gensim """
    proc_result=[]
    for c in document :
        doc=gensim.utils.simple_preprocess(c)
        proc_result.append(doc)
    return proc_result

def train_word2vec(train_data,model_name="word2vec_model"):
    """ Trains a word2vec model on the preprocessed data and saves it . """
    if not train_data:
        print("no training data")
        return
    #nlp = spacy.load('en')
    #train_data = [nlp(doc) for doc in train_data]
    print('data is ready for processing')
    w2v_corpus = gensim_preproc(train_data)
    cores = multiprocessing.cpu_count()
    model = Word2Vec(w2v_corpus, workers = cores-1,iter=30, size=150, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=20)
    model.save(model_name)
    print("Model Created Successfully")

if __name__ == "__main__":
    train_data = get_data('./data/training_data')
    print('data has been read successfully')
    train_word2vec(train_data)

data has been read successfully


In [12]:
model = Word2Vec.load('./models/word2vec_model_window7')

INFO - 14:56:46: loading Word2Vec object from ./models/word2vec_model_window7
INFO - 14:56:48: loading wv recursively from ./models/word2vec_model_window7.wv.* with mmap=None
INFO - 14:56:48: setting ignored attribute vectors_norm to None
INFO - 14:56:48: loading vocabulary recursively from ./models/word2vec_model_window7.vocabulary.* with mmap=None
INFO - 14:56:48: loading trainables recursively from ./models/word2vec_model_window7.trainables.* with mmap=None
INFO - 14:56:48: setting ignored attribute cum_table to None
INFO - 14:56:48: loaded ./models/word2vec_model_window7


In [29]:
model.most_similar('rpa')

C:\Users\DELL\Anaconda3\envs\retinanet\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('uipath', 0.6568434238433838),
 ('robotic', 0.44815418124198914),
 ('prism', 0.4274270236492157),
 ('predator', 0.4224933087825775),
 ('robotics', 0.4060060679912567),
 ('automation', 0.3704923987388611),
 ('jpl', 0.363048255443573),
 ('piloted', 0.3575410544872284),
 ('worksoft', 0.3563433289527893),
 ('robot', 0.35209590196609497)]